In [1]:
import pandas as pd

property_assess = pd.read_csv('assessment_per_capita.csv')

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3018: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
property_assess.columns

Index(['Unnamed: 0', 'acc_num', 'nb', 'nb_id', 'class', 'value', 'suite',
       'number', 'street', 'garage', 'lat', 'long', 'zoning', 'lot_size',
       'year_built', 'Assault', 'Break and Enter', 'Homicide', 'Robbery',
       'Sexual Assaults', 'Theft From Vehicle', 'Theft Of Vehicle',
       'Theft Over $5000', 'population', 'num_incidents', 'crime_per_capita'],
      dtype='object')

In [3]:
print(property_assess['lat'].mean(), property_assess['long'].mean())

53.52064058590727 -113.50898651489592


In [4]:
import folium

m = folium.Map(location=[property_assess['lat'].mean(), property_assess['long'].mean()], zoom_start=10)
m

In [5]:
def add_marker(row):
    if row['value'] < 200000:
        color = '#0061ff'
    elif row['value'] < 300000:
        color = '#0252d6'
    elif row['value'] < 400000:
        color = '#033b99'
    elif row['value'] < 500000:
        color = '#012668'
    elif row['value'] < 600000:
        color = '##00173f'
    else:
        color = '#820000'
    
    row_cur = '${:0,.2f}'.format(row['value'])        
    tooltip = '<p><b>Address:</b></p><br><p>' + str(row['number']) + " " + row['street'] \
              + '</p><br>' \
              + '<p><b>Assessed Value:</b></p><br><p>' + row_cur + '</p>'
    marker = folium.CircleMarker([row['lat'], row['long']], radius=2, 
                                     color=color, tooltip=tooltip)
    return marker

In [6]:
def nb_map(neighbourhood):
    my_nb = property_assess[property_assess['nb'] == str(neighbourhood).upper()]

    m2 = folium.Map(location=[my_nb['lat'].mean(), my_nb['long'].mean()], zoom_start=15)
    
    def add_marker(row):
        if row['value'] < 100000:
            color = '#bfd7ff'
        elif row['value'] < 200000:
            color = '#a4c4fc'
        elif row['value'] < 300000:
            color = '#89b4ff'
        elif row['value'] < 400000:
            color = '#669dff'
        elif row['value'] < 500000:
            color = '#3f84ff'
        elif row['value'] < 600000:
            color = '#1e6fff'
        elif row['value'] < 700000:
            color = '#005bff'
        elif row['value'] < 800000:
            color = '#0045c6'
        elif row['value'] < 900000:
            color = '#012f87'
        elif row['value'] < 1000000:
            color = '#002263'
        elif row['value'] < 1100000:
            color = '#001338'
        elif row['value'] < 120000:
            color = '#820000'
        elif row['value'] < 1300000:
            color = '#b70000'
        elif row['value'] < 1400000:
            color = '#db0000'
        else:
            color = '#ff0000'

        row_cur = '${:0,.2f}'.format(row['value'])        
        tooltip = '<p><b>Address:</b></p><p>' + str(row['number']) + " " + row['street'] \
                  + '</p>' \
                  + '<p><b>Assessed Value:</b></p><p>' + row_cur + '</p>'
        marker = folium.CircleMarker([row['lat'], row['long']], radius=2, 
                                         color=color, tooltip=tooltip)
        marker.add_to(m2)

    my_nb.apply(add_marker, axis=1)

    return m2

In [7]:
nb_map('starling')

In [8]:
nb_list = list(property_assess['nb'].unique())
nb_list = sorted(nb_list)

In [9]:
nb_map('WINDSOR PARK')

In [10]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [11]:
_ = interact(nb_map, neighbourhood=nb_list)

interactive(children=(Dropdown(description='neighbourhood', options=('ABBOTTSFIELD', 'ALBANY', 'ALBERTA AVENUE…